# CGC PSU with Base Class Testing

This notebook tests the new PSU class structure with PSUBase and PSU (with logging).

## Test Sections:
1. **Connection Testing**: Connect to PSU device
2. **Individual HK Functions**: Test individual housekeeping functions  
3. **Manual HK Cycle**: Execute one housekeeping cycle manually
4. **HK Thread**: Test automatic housekeeping thread

In [1]:
# Import required modules
import sys
import os
import logging

# Add src to path
sys.path.append(os.path.join(os.getcwd(), '..', '..', 'src'))

from devices.cgc.psu.psu import PSU

## Section 1: Connection Testing

Test PSU creation and connection with internal logger (automatic file creation).

In [ ]:
# Create PSU instance with internal logger
psu_internal = PSU("test_psu_internal", com=6, port=1)

print("PSU created with internal logger")
print(f"Device ID: {psu_internal.device_id}")
print(f"Logger: {psu_internal.logger.name}")
print(f"Status: {psu_internal.get_status()}")

PSU created with internal logger
Device ID: test_psu_internal
Logger: PSU_test_psu_internal_20251105_115512


In [ ]:
# Connect to device (will be logged)
print("Attempting to connect...")
result = psu_internal.connect()
print(f"Connection result: {result}")
print(f"Updated status: {psu_internal.get_status()}")

Attempting to connect...
Connection result: True


In [ ]:
## Section 2: Individual HK Functions Testing

Test each individual housekeeping function to see the structured logging output.

True

In [ ]:
# Test individual housekeeping functions
print("Testing individual housekeeping functions:")

# Test product info
print("\n1. Product Info:")
result = psu_internal._hk_product_info()
print(f"   Result: {result}")

# Test main state  
print("\n2. Main State:")
result = psu_internal._hk_main_state()
print(f"   Result: {result}")

# Test device state
print("\n3. Device State:")
result = psu_internal._hk_device_state()
print(f"   Result: {result}")

# Test general housekeeping
print("\n4. General Housekeeping:")
result = psu_internal._hk_general_housekeeping()
print(f"   Result: {result}")

(0, 19.54, 19.06, 18.94)

In [ ]:
# Test sensor and PSU-specific housekeeping functions
print("Testing PSU-specific housekeeping functions:")

# Test sensor data
print("\n5. Sensor Data:")
result = psu_internal._hk_sensor_data()
print(f"   Result: {result}")

# Test PSU0 ADC housekeeping
print("\n6. PSU0 ADC Housekeeping:")
result = psu_internal._hk_psu0_adc_housekeeping()
print(f"   Result: {result}")

# Test PSU1 ADC housekeeping
print("\n7. PSU1 ADC Housekeeping:")
result = psu_internal._hk_psu1_adc_housekeeping()
print(f"   Result: {result}")

# Test PSU0 housekeeping
print("\n8. PSU0 Housekeeping:")
result = psu_internal._hk_psu0_housekeeping()
print(f"   Result: {result}")

True

In [ ]:
# Test remaining PSU housekeeping functions
print("Testing remaining PSU housekeeping functions:")

# Test PSU1 housekeeping
print("\n9. PSU1 Housekeeping:")
result = psu_internal._hk_psu1_housekeeping()
print(f"   Result: {result}")

# Test PSU0 data
print("\n10. PSU0 Data:")
result = psu_internal._hk_psu0_data()
print(f"   Result: {result}")

# Test PSU1 data
print("\n11. PSU1 Data:")
result = psu_internal._hk_psu1_data()
print(f"   Result: {result}")

print("\n✓ All individual housekeeping functions tested!")
print("Check the log file for structured output with individual value logging.")

In [ ]:
## Section 3: Manual HK Cycle

Test executing one complete housekeeping cycle manually using the new hk_monitor() method.

<Logger PSU_test_psu_internal_20251105_112505 (INFO)>

In [ ]:
# Test manual housekeeping cycle
print("Testing manual housekeeping cycle:")
print("This will execute ALL housekeeping functions in one call...")

# Execute one complete housekeeping cycle
print("\nExecuting psu_internal.hk_monitor()...")
psu_internal.hk_monitor()

print("✓ Manual housekeeping cycle completed!")
print("Check the log file - you should see all structured logging output from all HK functions.")


Testing method calls with automatic logging:
0 0x8004 STATE_ERR_ILOCK


In [ ]:
# Alternative: Test single housekeeping cycle using do_housekeeping_cycle()
print("Testing single housekeeping cycle (alternative method):")

# First enable housekeeping (but don't start the thread)
psu_internal.hk_running = True

# Execute one cycle
print("\nExecuting psu_internal.do_housekeeping_cycle()...")
result = psu_internal.do_housekeeping_cycle()
print(f"Cycle result: {result}")

# Clean up
psu_internal.hk_running = False

print("✓ Alternative housekeeping cycle completed!")

Device enabled: False


In [ ]:
## Section 4: HK Thread Testing

Test the automatic housekeeping thread functionality.

True

In [ ]:
# Test automatic housekeeping thread
import time

print("Testing automatic housekeeping thread:")
print("Starting housekeeping with 3-second interval...")

# Start housekeeping thread
success = psu_internal.start_housekeeping(interval=3)
print(f"Start result: {success}")

if success:
    print(f"Housekeeping status: {psu_internal.get_status()}")
    print("\nLet it run for ~10 seconds to see multiple cycles...")
    print("Watch the log file for automatic periodic logging!")
    
    # Let it run for a bit
    time.sleep(10)
    
    print("\n✓ Housekeeping has been running for 10 seconds")
    print("Check your log file - you should see periodic housekeeping entries every 3 seconds!")

In [ ]:
# Stop housekeeping and disconnect
print("Stopping housekeeping and disconnecting...")

# Stop the housekeeping thread
stop_result = psu_internal.stop_housekeeping()
print(f"Stop housekeeping result: {stop_result}")

# Disconnect from device
disconnect_result = psu_internal.disconnect()
print(f"Disconnect result: {disconnect_result}")

print(f"\nFinal status: {psu_internal.get_status()}")
print("✓ Test completed! Check the log files in debugging/logs/")
print(f"Logger used: {psu_internal.logger.name}")